BLOCKGAME

[algospot의 BLOCKGAME 링크](https://algospot.com/judge/problem/read/BLOCKGAME)
  
### 문제  
시티빌과 비주얼드에 지친 진호와 현환이는 집에 굴러다니는 블럭들을 모아 새로운 게임을 하기로 했습니다. 5×5 크기의 게임판에서 시작해, 둘이 번갈아 가며 블럭을 하나씩 게임판에 내려놓습니다. 블럭은 L 모양으로 구성된 3칸짜리 블럭과 2칸짜리 블럭이 있으며, 항상 게임판에 있는 줄에 맞춰 내려놓아야 합니다. 블럭들은 서로 겹칠 수 없습니다. 다음 그림은 진행중인 게임판의 모습을 보여줍니다.  
![](http://algospot.com/media/judge-attachments/065ff270c832fbe7dbc54af1227ddd5b/midgame.png)  
그림에서 보이는 바와 같이 각 블록은 자유롭게 뒤집거나 회전해서 올려놓을 수 있습니다. 두 사람이 번갈아가며 블록을 올려놓다가 더 올려놓을 수 없게 되면 마지막에 올려놓은 사람이 승리합니다. 진행 중인 게임판이 주어질 때 이번 차례인 사람이 승리할 수 있는 방법이 있는지를 판단하는 프로그램을 작성하세요.  
     
### 출력   
이번 차례인 사람이 항상 이길 수 있는 방법이 있다면 WINNING을, 항상 질 수밖에 없다면 LOSING을 출력합니다.  

solution의  
game_map : 5X5 크기 게임판  

In [1]:
def solution(game_map):
    return "WINNING" if block_game(game_map) == 1 else "LOSING"
    # 1을 반환시 이기고, -1의 경우 지는 것이다.

def block_game(game_map, mem=None, state=None, num_of_dot= 0):
    if mem is None: # 메모이재이션 및 인덱스 초기화
        mem = [0 for _ in range(1 << 25)]
        state = ''
        for i in game_map:
            for j in i:
                if j == '#':
                    state += '1'
                else:
                    state += '0'
                    num_of_dot += 1
        # '.'의 경우 0, '#'의 경우 1을 마킹해서 25자리의 2진수로 맵을 표현한다.
        state = int(state, 2) # 이진표현 state의 10진수 정수화
    if mem[state] != 0: # 해당 상태가 이미 계산된 경우
        return mem[state] # 계산된 값을 반환
    if num_of_dot < 2: # 만약 남은 칸 수가 2 이하라면
        mem[state] = -1 # 현재 턴의 사람이 진것이다.
        return mem[state] # 졌다는 -1값을 반환
    
    answer = -1 # 일단 질 경우로 답을 초기화
    for i in range(5):
        if answer == 1: break # 만약 이길 수 있는 방법이 있다면 바로 for문을 나간다.
        for j in range(5):
            if i>0 and j<4 and check_null(state, i-1, j) and check_null(state, i, j) and check_null(state, i, j+1):
                new_state = state | ((1 << (i-1)*5+j) + (1 << (i)*5+j) + (1 << (i)*5+(j+1)))
                answer = max(answer, -block_game(game_map, mem, new_state, num_of_dot-3))
                if answer == 1: break
            if j<4 and i<4 and check_null(state, i, j+1) and check_null(state, i, j) and check_null(state, i+1, j):
                new_state = state | ((1 << (i)*5+(j+1)) + (1 << (i)*5+j) + (1 << (i+1)*5+(j)))
                answer = max(answer, -block_game(game_map, mem, new_state, num_of_dot-3))
                if answer == 1: break
            if i<4 and j>0 and check_null(state, i+1, j) and check_null(state, i, j) and check_null(state, i, j-1):
                new_state = state | ((1 << (i+1)*5+(j)) + (1 << (i)*5+j) + (1 << (i)*5+(j-1)))
                answer = max(answer, -block_game(game_map, mem, new_state, num_of_dot-3))
                if answer == 1: break
            if j>0 and i>0 and check_null(state, i, j-1) and check_null(state, i, j) and check_null(state, i-1, j):
                new_state = state | ((1 << (i)*5+(j-1)) + (1 << (i)*5+j) + (1 << (i-1)*5+(j)))
                answer = max(answer, -block_game(game_map, mem, new_state, num_of_dot-3))
                if answer == 1: break
            # L자의 3칸을 차지하는 블록에 대해서는 4가지 경우가 나오며, 해당 경우에 대해 재귀
            if i<4 and check_null(state, i, j) and check_null(state, i+1, j):
                new_state = state | ((1 << (i)*5+j) + (1 << (i+1)*5+(j)))
                answer = max(answer, -block_game(game_map, mem, new_state, num_of_dot-2))
                if answer == 1: break
            if j<4 and check_null(state, i, j) and check_null(state, i, j+1):
                new_state = state | ((1 << (i)*5+j) + (1 << (i)*5+(j+1)))
                answer = max(answer, -block_game(game_map, mem, new_state, num_of_dot-2))
                if answer == 1: break
            # ㅡ자의 2칸을 차지하는 블록에 대해서는 2가지 경우가 나오며, 해당 경우에 대해 재귀
    mem[state] = answer # 답을 메모이재이션 리스트에 저장
    return answer # 그리고 답 반환

def check_null(state, i, j): # 해당 상태의 맵에서, (i,j) 좌표가 비어져있는지 체크
    if state - (state & (1 << (i*5+j))) == state: # 비트 연산을 통해서 해당 좌표 비었는지 체크
        return True # 비었다면 True
    else:
        return False # 차있다면 False

In [2]:
game_map = [[".",".",".",".","."],
            [".","#","#",".","."],
            ["#","#",".",".","#"],
            ["#",".","#","#","#"],
            [".",".","#",".","."]]

In [3]:
solution(game_map)

'WINNING'

In [4]:
game_map = [["#",".",".","#","#"],
            ["#","#",".","#","#"],
            ["#","#",".","#","#"],
            ["#",".",".",".","#"],
            ["#","#",".","#","#"]]

In [5]:
solution(game_map)

'WINNING'

In [6]:
game_map = [[".",".",".",".","."],
            [".",".",".",".","."],
            [".",".",".",".","."],
            [".",".",".",".","."],
            [".",".",".",".","."]]

In [7]:
solution(game_map) # 굉장히 오래 걸린다. -> 복잡도 더 줄일 것

'LOSING'